In [41]:
#importing libraries
import dlib
import cv2
import numpy as np
import face_recognition
import os
from scipy.spatial import distance
from imutils import face_utils
import imutils

In [3]:
# Functions Included :-
# Smile detection
# Facial Recognition
# Gaze Detection
# Air Canvas
# Drowsiness Detection
# Invisibility Cloak

In [5]:
# Smile detection
# XML Files for face, eyes and smile  haar cascade detector
face_cascade =cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml') 
smile_cascade = cv2.CascadeClassifier('haarcascade_smile.xml') 
# Function to call to detect smile
def smiledetect(frame): 
    ''' This function takes in the image frame as input either in video or image mode and returns a frame with 
    the face enclosed in blue and smile enclosed in red'''
    # Converting to gray scale for detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
    # Detecting multiple faces from the grayscale image this returns the corner locations of the face region of interest
    faces = face_cascade.detectMultiScale(gray, 1.3, 5) 
    for (x, y, w, h) in faces: 
        # Draw the blue colored rectangle around the face if detected
        cv2.rectangle(frame, (x, y), ((x + w), (y + h)), (255, 0, 0), 2) 
        # Taking only the region where face s found to process for smile 
        # this reduces the false positives as well as speeds up processing
        roi_gray = gray[y:y + h, x:x + w] 
        roi_color = frame[y:y + h, x:x + w] 
        # Detect smiles in the image
        smiles = smile_cascade.detectMultiScale(roi_gray, 1.8, 35) 
        # Draw a rectangle around the smile detected
        for (sx, sy, sw, sh) in smiles: 
            cv2.rectangle(roi_color, (sx, sy), ((sx + sw), (sy + sh)), (0, 0, 255), 2) 
    return frame 
https://www.geeksforgeeks.org/

In [61]:
# Facial Recognition ---------------------------------------------------------------------------------------#
# Creating directories
KNOWN_FACES_DIR = 'known_faces'     
UNKNOWN_FACES_DIR = 'unknown_faces'
TOLERANCE = 0.6
FRAME_THICKNESS = 3
FONT_THICKNESS = 2
MODEL = 'cnn'  # default: 'hog', other one can be 'cnn' - CUDA accelerated (if available) deep-learning pretrained model
# Creating empty lists to input known image encodings and their labels
known_faces = []
known_names = []
# We oranize known faces as subfolders of KNOWN_FACES_DIR Each subfolder's name becomes our label (name)
for name in os.listdir(KNOWN_FACES_DIR):
    # Next we load every file of faces of known person
    for filename in os.listdir(f'{KNOWN_FACES_DIR}/{name}'):
        # Load an image
        image = face_recognition.load_image_file(f'{KNOWN_FACES_DIR}/{name}/{filename}')
        # Get 128-dimension face encoding
        encoding = face_recognition.face_encodings(image)[0]
        # Append encodings and name
        known_faces.append(encoding)
        known_names.append(name)
# Now let's loop over a folder of faces we want to label
for filename in os.listdir(UNKNOWN_FACES_DIR):
    # Load image
    print(f'Filename {filename}', end='')
    image = face_recognition.load_image_file(f'{UNKNOWN_FACES_DIR}/{filename}')
    # This time we first grab face locations - we'll need them to draw boxes
    locations = face_recognition.face_locations(image, model=MODEL)
    # Now since we know loctions, we can pass them to face_encodings as second argument
    # Without that it will search for faces once again slowing down whole process
    encodings = face_recognition.face_encodings(image, locations)
    # We passed our image through face_locations and face_encodings, so we can modify it
    # First we need to convert it from RGB to BGR as we are going to work with cv2
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    # But this time we assume that there might be more faces in an image - we can find faces of dirrerent people
    print(f', found {len(encodings)} face(s)')
    for face_encoding, face_location in zip(encodings, locations):
        # We use compare_faces (but might use face_distance as well)
        # Returns array of True/False values in order of passed known_faces
        results = face_recognition.compare_faces(known_faces, face_encoding, TOLERANCE)
        # Since order is being preserved, we check if any face was found then grab index
        # then label (name) of first matching known face withing a tolerance
        match = None
        if True in results:  # If at least one is true, get a name of first of found labels
            match = known_names[results.index(True)]
            print(f' - {match} from {results}')
            # Each location contains positions in order: top, right, bottom, left
            cv2.putText(image, match, (face_location[3] + 10, face_location[2] + 15),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200, 200, 200), FONT_THICKNESS)
    # Show image
    cv2.imshow(filename, image)
    cv2.waitKey(0)
    cv2.destroyWindow(filename)
#--------------------------------end-------------------------------------------------------------------------------#
https://pythonprogramming.net/

In [ ]:
# Gaze Detection --------------------------------------------------------------------------------------------------#
import cv2
import dlib
import numpy as np
# Function to return the landmark coordinated in numpy array format
def shape_to_np(shape, dtype="int"):
    # initialize the list of (x, y)-coordinates
    coords = np.zeros((68, 2), dtype=dtype)
    # loop over the 68 facial landmarks and convert them to a 2-tuple of (x, y)-coordinates
    for i in range(0, 68):
        coords[i] = (shape.part(i).x, shape.part(i).y)
    return coords
# To add the eye region ot the mask
def eye_on_mask(mask, side):
    # selecting the list containing points in the side array
    points = [shape[i] for i in side]
    points = np.array(points, dtype=np.int32)
    # This function makes a convex polygon shape with the given points and sets those coordinate values in mask to 255
    mask = cv2.fillConvexPoly(mask, points, 255)
    return mask
# To draw the red color dot at the center of eyeball
def contouring(thresh, mid, img, right=False):
    # This returns the contours ie. areas with similar colour texture 
    cnts, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    try:
        cnt = max(cnts, key = cv2.contourArea) # finding the contour with max are ie. the eyeball
        # The folowwing lines are used to detect the movements of the contour area
        M = cv2.moments(cnt)
        cx = int(M['m10']/M['m00'])
        cy = int(M['m01']/M['m00'])
        # To ensure midpoint stays in same area to remove random detections or objects other than eyeball
        if right:
            cx += mid
        cv2.circle(img, (cx, cy), 4, (0, 0, 255), 2)
    except:
        pass
# These are used to get the face locations in 68 points using deep learning library dlib
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# These are the numbers of the points in the face where left and right eye is matched by dlib 
left = [36, 37, 38, 39, 40, 41]
right = [42, 43, 44, 45, 46, 47]

cap = cv2.VideoCapture(0)
ret, img = cap.read()
thresh = img.copy()

cv2.namedWindow('image')
# The smaller the kernel more stricter the detection more the chances of false negatives
kernel = np.ones((9, 9), np.uint8)

def nothing(x):
    pass
# Uncomment the following line for gui application to manually select the threshold
#cv2.createTrackbar('threshold', 'image', 0, 255, nothing)

# The infinite loop for video capture
while(True):
    ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # detects the region of interest where face is found
    rects = detector(gray, 1)
    for rect in rects:
        # For each region where face is found, the following lines predict the 68 locations corresponding to each face
        shape = predictor(gray, rect)
        shape = shape_to_np(shape)
        # Creating a mask and making only the region with eyes as white
        mask = np.zeros(img.shape[:2], dtype=np.uint8)
        mask = eye_on_mask(mask, left)
        mask = eye_on_mask(mask, right)
        # Image enhancement operations to make the image smoother and easyily detectable
        mask = cv2.dilate(mask, kernel, 5)
        eyes = cv2.bitwise_and(img, img, mask=mask)
        mask = (eyes == [0, 0, 0]).all(axis=2)
        eyes[mask] = [255, 255, 255]
        mid = (shape[42][0] + shape[39][0]) // 2
        eyes_gray = cv2.cvtColor(eyes, cv2.COLOR_BGR2GRAY)
        # For simplicity we assume a standard threshold value we may use gui trackbar by uncommenting the following line
        #threshold = cv2.getTrackbarPos('threshold', 'image')
        threshold = 45
        _, thresh = cv2.threshold(eyes_gray, threshold, 255, cv2.THRESH_BINARY)
        # Image enhacement for the final image as well
        thresh = cv2.erode(thresh, None, iterations=2) #1
        thresh = cv2.dilate(thresh, None, iterations=4) #2
        thresh = cv2.medianBlur(thresh, 3) #3
        thresh = cv2.bitwise_not(thresh)
        contouring(thresh[:, 0:mid], mid, img)
        contouring(thresh[:, mid:], mid, img, True)
        #for (x, y) in shape[36:48]:
        #     cv2.circle(img, (x, y), 2, (255, 0, 0), -1)
    # show the image with the face detections + facial landmarks
    cv2.imshow('eyes', img)
    cv2.imshow("image", thresh)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()
#--------------------------------------------------end-----------------------------------------------------------#
https://towardsdatascience.com/real-time-eye-tracking-using-opencv-and-dlib-b504ca724ac6

In [ ]:
# Drowsiness detection ------------------------------------------------------------------------------------------#
# One can also deploy the flask application named inv.py to verify this application
# Setting up some values 
thresh = 0.25
frame_check = 50
detect = dlib.get_frontal_face_detector()
predict = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
# Finding the starting and ending values of eye points, can hardcode it as earlier
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

# Fuction to find the ratio of length to width of the eye, this helps determine if eye is closed or not
def eye_aspect_ratio(eye):
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    C = distance.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear
# Loop to determine sleeping or not
cap=cv2.VideoCapture(0)
flag=0
while True:
    ret, frame=cap.read()
    frame = imutils.resize(frame, width=450)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    subjects = detect(gray, 0)
    for subject in subjects:
        # Predict the landmarks using dlib library
        shape = predict(gray, subject)
        shape = face_utils.shape_to_np(shape)#converting to NumPy Array
        # The list of points of the left and right eye
        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]
        # Finding the aspect ratio of eyes to determine if they are open or close
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)
        ear = (leftEAR + rightEAR) / 2.0
        # Draw a line around the eyes
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
        # If eye is detected closed an alert is shown on screen, one can use audio to ring a bell too
        if ear < thresh:
            flag += 1
            if flag >= frame_check:
                cv2.putText(frame, "****************ALERT!****************", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                cv2.putText(frame, "****************ALERT!****************", (10,325),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        else:
            flag = 0
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        cv2.destroyAllWindows()
        cap.release()
        break
#----------------------------------------------End----------------------------------------------------------------#
https://towardsdatascience.com/

In [ ]:
# nvisibility Cloak ---------------------------------------------------------------------------------------------#
def invisible(image,background):
    img = np.flip(image, axis = 1)  #return_val, img = capture_video.read() 
    background = np.flip(background, axis = 1) # flipping of the frame 
    # convert the image - BGR to HSV  as we focused on detection of red color converting BGR to HSV for better 
    # detection or you can convert it to gray 
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV) 
    # ranges should be carefully chosen setting the lower and upper range for mask1 
    lower_red = np.array([100, 40, 40])	 
    upper_red = np.array([100, 255, 255]) 
    mask1 = cv2.inRange(hsv, lower_red, upper_red) 
    # setting the lower and upper range for mask2 
    lower_red = np.array([155, 40, 40]) 
    upper_red = np.array([180, 255, 255]) 
    mask2 = cv2.inRange(hsv, lower_red, upper_red) 
    mask1 = mask1 + mask2 
    # Refining the mask corresponding to the detected red color 
    mask1 = cv2.morphologyEx(mask1, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8), iterations = 2) 
    mask1 = cv2.dilate(mask1, np.ones((3, 3), np.uint8), iterations = 1) 
    mask2 = cv2.bitwise_not(mask1) 
    # Generating the final output 
    res1 = cv2.bitwise_and(background, background, mask = mask1) 
    res2 = cv2.bitwise_and(img, img, mask = mask2) 
    final_output = cv2.addWeighted(res1, 1, res2, 1, 0)
    return final_output
#--------------------------------------------------End----------------------------------------------------------#
https://www.geeksforgeeks.org/